In [1]:
# This code is part of Qiskit.
#
# (C) Copyright IBM 2021.
#
# This code is licensed under the Apache License, Version 2.0. You may
# obtain a copy of this license in the LICENSE.txt file in the root directory
# of this source tree or at http://www.apache.org/licenses/LICENSE-2.0.
#
# Any modifications or derivative works of this code must retain this
# copyright notice, and modified files need to carry a notice indicating
# that they have been altered from the originals.

"""Test Rabi amplitude Experiment class."""
import sys
sys.path.insert(0,"/home/rupeshknn/projects/qc/pulse-backend-qamp22/qiskit-experiments/test/base.py")
from base import QiskitExperimentsTestCase
# from qiskit_experiments.test.base import QiskitExperimentsTestCase
import unittest
import numpy as np

from qiskit import QuantumCircuit, transpile
from qiskit.exceptions import QiskitError
from qiskit.circuit import Parameter
from qiskit.providers.basicaer import QasmSimulatorPy
from qiskit.qobj.utils import MeasLevel
import qiskit.pulse as pulse

from qiskit_experiments.framework import ExperimentData, ParallelExperiment
from qiskit_experiments.library import Rabi, EFRabi

from qiskit_experiments.curve_analysis.standard_analysis.oscillation import OscillationAnalysis
from qiskit_experiments.data_processing.data_processor import DataProcessor
from qiskit_experiments.data_processing.nodes import Probability
from qiskit_experiments.test.mock_iq_backend import MockIQBackend
from qiskit_experiments.test.mock_iq_helpers import MockIQRabiHelper as RabiHelper
from qiskit_experiments.framework.experiment_data import ExperimentStatus

In [ ]:
from qiskit_experiments.test.iq_pulse_backend import SingleTransmonTestBackend

In [ ]:
import pickle

In [25]:
pulse_backend = SingleTransmonTestBackend()

In [73]:
pulse_backend._defaults.cmd_def[0].sequence[0].parameters['amp']#.to_dict()#.to_dict()

(0.1+0j)

In [22]:
class TestRabiEndToEnd(QiskitExperimentsTestCase):
    """Test the rabi experiment."""

    def setUp(self):
        """Setup the tests."""
        super().setUp()

        self.qubit = 1

        with pulse.build(name="x") as sched:
            pulse.play(pulse.Drag(160, Parameter("amp"), 40, 0.4), pulse.DriveChannel(self.qubit))

        self.sched = sched

    # pylint: disable=no-member
    def test_rabi_end_to_end(self):
        """Test the Rabi experiment end to end."""

        test_tol = 0.01
#         rabi_experiment_helper = RabiHelper()
#         backend = MockIQBackend(rabi_experiment_helper)
        backend = SingleTransmonTestBackend()

        rabi = Rabi(self.qubit, self.sched)
        rabi.set_experiment_options(amplitudes=np.linspace(-0.95, 0.95, 21))
        expdata = rabi.run(backend)
        
        outfile = open('rabi_ete_test_expdata','wb')
        pickle.dump(expdata, outfile)
        outfile.close()
        
        self.assertExperimentDone(expdata)
        result = expdata.analysis_results(0)

        self.assertEqual(result.quality, "good")
        # The comparison is made against the object that exists in the backend for accurate testing
        self.assertAlmostEqual(
            result.value.params["freq"], , delta=test_tol
        )

#         # updating 'amplitude_to_angle' parameter in the experiment helper
# #         rabi_experiment_helper.amplitude_to_angle = np.pi / 2

#         expdata = rabi.run(backend)
#         self.assertExperimentDone(expdata)
#         result = expdata.analysis_results(0)
#         self.assertEqual(result.quality, "good")
#         self.assertAlmostEqual(
#             result.value.params["freq"], backend.experiment_helper.rabi_rate(), delta=test_tol
#         )

#         # updating 'amplitude_to_angle' parameter in the experiment helper and experiment options
# #         rabi_experiment_helper.amplitude_to_angle = 2.5 * np.pi
#         rabi.set_experiment_options(amplitudes=np.linspace(-0.95, 0.95, 101))

#         expdata = rabi.run(backend)
#         self.assertExperimentDone(expdata)
#         result = expdata.analysis_results(0)
#         self.assertEqual(result.quality, "good")
#         self.assertAlmostEqual(
#             result.value.params["freq"], backend.experiment_helper.rabi_rate(), delta=test_tol
#         )

    def test_wrong_processor(self):
        """Test that we can override the data processing by giving a faulty data processor."""
        backend = MockIQBackend(RabiHelper())
        rabi = Rabi(self.qubit, self.sched)
        fail_key = "fail_key"

        rabi.analysis.set_options(data_processor=DataProcessor(fail_key, []))
        # pylint: disable=no-member
        rabi.set_run_options(shots=2)
        data = rabi.run(backend)
        result = data.analysis_results()

        self.assertEqual(data.status(), ExperimentStatus.ERROR)
        self.assertEqual(len(result), 0)

    def test_experiment_config(self):
        """Test converting to and from config works"""
        exp = Rabi(0, self.sched)
        loaded_exp = Rabi.from_config(exp.config())
        self.assertNotEqual(exp, loaded_exp)
        self.assertTrue(self.json_equiv(exp, loaded_exp))

    @unittest.skip("Schedules are not yet JSON serializable")
    def test_roundtrip_serializable(self):
        """Test round trip JSON serialization"""
        exp = Rabi(0, self.sched)
        self.assertRoundTripSerializable(exp, self.json_equiv)

In [23]:
# suite = unittest.makeSuite(TestRabiEndToEnd, 'test')
# result = unittest.TestResult()
# suite.run(result)


obj = TestRabiEndToEnd("test_rabi_end_to_end")
obj.setUp()

In [24]:
obj.test_rabi_end_to_end()

AttributeError: 'SingleTransmonTestBackend' object has no attribute 'experiment_helper'